# Introduction
Previously, the majority of transport reactions were removed as there were quite some incorrect reactions present. However, now it is worth revisiting if we should reintroduce some transports. 

For example, various amino acids can also be produced and exported in a sort of overflow metabolism mechanism. Now this is prevented because there is no transport reaction. So here I will go through the list of amino acid transports and check what type of transport should be added and add them if it makes sense. 


In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Metabolite, Reaction

In [3]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [5]:
df = pd.read_csv('../databases/transport_aa.csv', sep = ';')

In [6]:
df[0:5]

,Amino acid,Transport type,Direction,ID
0,gly_c,sym_proton,in,GLYt
1,gly_c,passive,out,GLYt2
2,ala__L_c,passive,out,ALAt2
3,ala__L_c,sym_proton,in,ALAt
4,val__L_c,sym_proton,rev,VALt


In [7]:
#add all extracellular metabolites
for index,react in df.iterrows():
    met = react['Amino acid']
    ex_met_id = met[:-2] + '_e'
    model.add_metabolites(Metabolite(id=ex_met_id))
    model.metabolites.get_by_id(ex_met_id).name = model.metabolites.get_by_id(met).name
    model.metabolites.get_by_id(ex_met_id).formula = model.metabolites.get_by_id(met).formula
    model.metabolites.get_by_id(ex_met_id).charge = model.metabolites.get_by_id(met).charge
    model.metabolites.get_by_id(ex_met_id).annotation = model.metabolites.get_by_id(met).annotation
    model.metabolites.get_by_id(ex_met_id).compartment = 'e'

In [8]:
#add all reactions to the model
for index,react in df.iterrows():
    rct = react['ID']
    model.add_reaction(Reaction(id=rct))
    c_met = react['Amino acid']
    e_met = c_met[:-2] + '_e'
    model.reactions.get_by_id(rct).name = 'Transport of ' + model.metabolites.get_by_id(c_met).name 
    model.reactions.get_by_id(rct).annotation['sbo'] = 'SBO:0000185'
    t_type = react['Transport type']
    direction = react['Direction']
    if t_type in 'passive':
        model.reactions.get_by_id(rct).add_metabolites({
            model.metabolites.get_by_id(c_met): -1,
            model.metabolites.get_by_id(e_met): 1
        })
        if direction in 'rev':
            model.reactions.get_by_id(rct).bounds = (-1000,1000)
        elif direction in 'out':
            model.reactions.get_by_id(rct).bounds = (0,1000)
        elif direction in 'in':
            model.reactions.get_by_id(rct).bounds = (-1000,0)
        else:
            print (rct)
    elif t_type in 'sym_proton':
        model.reactions.get_by_id(rct).add_metabolites({
            model.metabolites.get_by_id(c_met): -1,
            model.metabolites.get_by_id(e_met): 1,
            model.metabolites.h_c:-1,
            model.metabolites.h_e: 1
        })
        if direction in 'rev':
            model.reactions.get_by_id(rct).bounds = (-1000,1000)
        elif direction in 'out':
            model.reactions.get_by_id(rct).bounds = (0,1000)
        elif direction in 'in':
            model.reactions.get_by_id(rct).bounds = (-1000,0)
        else:
            print (rct)
    elif t_type in 'anti_proton':
        model.reactions.get_by_id(rct).add_metabolites({
            model.metabolites.get_by_id(c_met): 1,
            model.metabolites.get_by_id(e_met): -1,
            model.metabolites.h_c:-1,
            model.metabolites.h_e: 1
        })
        if direction in 'rev':
            model.reactions.get_by_id(rct).bounds = (-1000,1000)
        elif direction in 'out':
            model.reactions.get_by_id(rct).bounds = (-1000,0)
        elif direction in 'in':
            model.reactions.get_by_id(rct).bounds = (0,1000)
        else:
            print (rct)
    elif t_type in 'abc':
        model.reactions.get_by_id(rct).add_metabolites({
            model.metabolites.get_by_id(c_met): 1,
            model.metabolites.get_by_id(e_met): -1,
            model.metabolites.atp_c:-1,
            model.metabolites.h2o_c:-1,
            model.metabolites.adp_c:1,
            model.metabolites.h_c:1,
            model.metabolites.pi_c:1            
        })
        if direction in 'rev':
            model.reactions.get_by_id(rct).bounds = (-1000,1000)
        elif direction in 'out':
            model.reactions.get_by_id(rct).bounds = (-1000,0)
        elif direction in 'in':
            model.reactions.get_by_id(rct).bounds = (0,1000)
        else:
            print (rct)
    else:
        print(rct.id)
    #add exchange where they are not present already
    ex_id = 'EX_'+ e_met
    try: 
        model.add_boundary(model.metabolites.get_by_id(e_met),type = 'exchange', reaction_id = ex_id, sbo_term = 'SBO:0000627')
        model.reactions.get_by_id(ex_id).bounds = (0,1000)
    except ValueError: #for the exchanges already present, just be sure bounds only allow production
        model.reactions.get_by_id(ex_id).bounds = (0,1000)

In [9]:
#check number of rcts added
len(model.reactions)

979

In [10]:
model.optimize().objective_value

1.7026543554461835

When we have removed the glutathionine uptake from the model, this is quite realistic and correct. 

With gthrd still in the biomass: We see we get huge production of cysteine: this is because our model can still take up gthrd which is not just converted and expelled as cysteine which is quite unrealistic. Additionally, we get huge amounts of aspartate being produced. This is because the gthrd is also converted into glutamate which is then converted into aspartate and expelled. So getting rid of the gthrd should solve these issues. So when this has been fixed, we should re-check this script and commit it. 

In [12]:
solution = model.optimize()
for rct in model.exchanges:
    if solution[rct.id] == 0:
        continue
    else:
        print(rct.id, ': ', solution[rct.id])


EX_nh4_e :  -16.342345303658945
EX_glc__D_e :  -10.0
EX_thm_e :  -0.00037969192126450233
EX_fe2_e :  -0.0578954275597195
EX_h2o_e :  34.21261772250256
EX_o2_e :  -72.1047562153193
EX_co2_e :  -0.08369097578658383
EX_h_e :  21.044972969333834
EX_qh2_e :  -0.0005107963066338584
EX_nadp_e :  -0.013110438536948503
EX_Biomass_e :  1.7026543554461835
EX_so4_e :  -0.3652619971135759
EX_k_e :  -1.2058198145269872
EX_mg2_e :  -0.16686012683372597
EX_ca2_e :  -0.005363361219655478


In [13]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')